In [1]:
import os
import pandas as pd
import seaborn as sns

from dotenv import load_dotenv
from google.cloud import bigquery
from google.oauth2 import service_account

import matplotlib.pyplot as plt

load_dotenv()

_GCP_PROJECT_ID = os.getenv('GCP_PROJECT_ID')
_BQ_DATASET_MARTS = os.getenv('BIGQUERY_DATASET_MARTS')

# BigQuery Connection

In [2]:
try:
    creds = service_account.Credentials.from_service_account_info(
        st.secrets['gcp_service_account']
    )
    client = bigquery.Client(credentials=creds, project=creds.project_id)
except Exception:
    # Local dev fallback (requires GOOGLE_APPLICATION_CREDENTIALS)
    client = bigquery.Client()

# Data Loading

In [ ]:
# Get activities
query_activities = f"SELECT * FROM {_GCP_PROJECT_ID}.{_BQ_DATASET_MARTS}.fct_activities"
df_activities = client.query(query_activities).to_dataframe()

# Get activity streams -> Not needed for now
# query_activits_streams = f"SELECT * FROM {_GCP_PROJECT_ID}.{_BQ_DATASET_MARTS}.fct_activity_streams"
# df_activity_streams = client.query(query_activits_streams).to_dataframe()

In [ ]:
print(f'Count activities: {len(df_activities)}')
# print(f'Count stream pts: {len(df_activity_streams)}')

Count activities: 405
Count stream pts: 977480


In [12]:
df_activities.columns

Index(['activity_id', 'athlete_id', 'gear_id', 'activity_name', 'discipline',
       'start_date_local', 'activity_date_local', 'activity_year',
       'activity_month', 'activity_weekday', 'activity_hour_local',
       'distance_m', 'distance_km', 'moving_time_s', 'elapsed_time_s',
       'avg_pace_min_per_km', 'avg_speed_kph', 'max_speed_kph',
       'avg_speed_overall_kph', 'elevation_gain_m', 'avg_heartrate',
       'max_heartrate', 'avg_cadence', 'energy_kj', 'avg_watts', 'max_watts',
       'weighted_watts', 'kudos_count', 'comment_count', 'achievement_count',
       'is_commute', 'is_trainer', 'has_heartrate', 'map_id', 'map_polyline',
       'mart_loaded_at'],
      dtype='object')

# Filter relevant data

In [5]:
# Filter
df_activities_filtered = df_activities[
    (df_activities['discipline'] == 'Run')
    & (df_activities['is_trainer'] == False)
].copy()
print(f'Len relevant activities: {len(df_activities_filtered)}')

Len relevant activities: 237


In [13]:
df_activities_filtered.head()

,activity_id,athlete_id,gear_id,activity_name,discipline,start_date_local,activity_date_local,activity_year,activity_month,activity_weekday,...,weighted_watts,kudos_count,comment_count,achievement_count,is_commute,is_trainer,has_heartrate,map_id,map_polyline,mart_loaded_at
162,14631802062,133094316,g20984891,4x4x48 Challenge #1,Run,2025-05-29 16:00:03+00:00,2025-05-29,2025,5,5,...,306.0,3,2,0,False,False,True,a14631802062,keluHmm|j@D|A?n@DjA@v@WbEKf@GjAMlAOz@i@zAOv@]`...,2026-01-29 05:02:21.760191+00:00
163,14647712197,133094316,g20984891,4x4x48 Challenge #11,Run,2025-05-31 08:00:10+00:00,2025-05-31,2025,5,7,...,296.0,3,0,0,False,False,True,a14647712197,keluHan|j@At@Fj@D|AEvBGdACp@Mv@MvAOtCI^m@vAaAb...,2026-01-29 05:02:21.760191+00:00
164,14649944715,133094316,g20984891,4x4x48 Challenge #12,Run,2025-05-31 12:00:08+00:00,2025-05-31,2025,5,7,...,308.0,4,0,0,False,False,True,a14649944715,meluHyo|j@BvAPtEATIx@EbAMdASv@?|@Gz@Mx@a@pB{@`...,2026-01-29 05:02:21.760191+00:00
165,14644126049,133094316,g20984891,4x4x48 Challenge #8,Run,2025-05-30 20:00:06+00:00,2025-05-30,2025,5,6,...,310.0,3,0,0,False,False,True,a14644126049,geluHmn|j@@tAFfB?|@MhBKjCOlAGlAQhA_@hAc@`Ba@jA...,2026-01-29 05:02:21.760191+00:00
166,15588726735,133094316,g20984891,5k Max,Run,2025-08-26 07:41:44+00:00,2025-08-26,2025,8,3,...,386.0,2,0,14,False,False,True,a15588726735,ehypHqvet@JnAFb@DtA?jDG~B@|@CfBB|@WhHE`GI~CCtE...,2026-01-29 05:02:21.760191+00:00
